# このNotebookについて
- このNotebookは、Riiidを解く前に知っておくとよいものをまとめたものです。
- 必要と思ったものがあれば随時更新します(一部、作成中のところもありますがご了承ください)
- 筆者は、英語が得意ではないため、誤り翻訳、誤解などが発生する可能性があります。

# Riiidコンペについて

## コンペ主催者
- [RiiidLabs](https://riiidlabs.ai/solutions)
    - AIを活用して、あらゆる業界の問題解決をする企業のようです。現在は、教育問題に取り組んでいるようだ。
    - 以下の情報より、Riiidは、AIによって個人最適化されたTOEIC学習アプリを提供する韓国初スタートアップとのこと
    - https://sunryse.co/app/startups/r_6WjOnZ6mV7JPbpY
    
## 知っておくと戦いやすくなる知識(のはず)

- ここの項目に書くことは推測です。
- [riiid](https://www.riiid.co/en/product)からリンクをたどると[TOEIC学習アプリ・santatoeic](https://santatoeic.jp/)にたどり着きます。
- santatoeicは登録無料である程度、テストや講義を試すことができます。
- ためした結果、実は、今回のコンペのデータ構成と、santatoeicを比較するとかなり類似しているように思えます。
- 類似している箇所を気づいた限り挙げると以下のとおりです
    - lectures.csvのpartの数が７つと、santatoeiと一致する
    - questions.csvのpartは、TOEICテストの関連セクション
    - santatoeicにも診断テスト、講義がある。
    - santatoeicは、AIチューターシステムを採用している。
- つまり、今回のコンペはsantatoeicと深く関係があるかもしれません。
- よって、このコンペのデータ構成がイメージしづらいときは、santatoeicを使ってみるとイメージしやすくなります。
    
## 前提条件

- これ以降の説明は、学生達がTOEIC学習アプリを使って診断テスト、講義を受けている場面を想定して記載します。


## 主催者がコンペを開催する意図(目的)

- (注意)以下に書く内容は、筆者の解釈です。
- Riiid社は、学生のレベルに合わせたコンテンツ(質問、講義)を提供したい。
- それを実現するため、ユーザの使用状況(問題回答状況、正解率など)から、最適なコンテンツを提供を目指している。
- コンペのお題は、過去の正解状況などを元に、質問に答えられるかを予測します。
- 予測の目的は、AIチューターシステムが、学生のレベルに合わせたコンテンツを作るためと推測されます。



## コンペの概要

- Riiid! Answer Correctness Prediction
    - 生徒は、次の問題に答えられるか！AIの力で予測してください
- Track knowledge states of 1M+ students in the wild
    - EdNet上の100万人以上の学生の知識状態(問題回答状況、正解率など)を追跡する

## コンペが開催される背景

現在、2億6000万人の子供たちが学校に通っていません。
これらの若い学生の半数以上は、最低限の読書と数学の基準を満たしていませんでした。
COVID-19のこともあり 現在の教育システムを再構築する必要がある。

## コンペでやるべきこと

生徒が将来の相互作用でどのように行動するかを正確に予測することです。 
RiiidのEdNetデータを使用して、機械学習スキルを活用して予測します。

そして、時間の経過に伴う学生の知識のモデリングである「Knowledge Tracing(知識追跡)」のアルゴリズムを作成することです。


受賞(優勝？)チームは、2021年2月に開催されるAI教育に関するAAAI-2021ワークショップ（AIによるCOVID後の教育の想像）でモデルを発表するよう招待されます。

# コンペで使用するデータ

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## train.csv(訓練データ)

In [ ]:
#データ多いため100万行まで読み込んで見る
train_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv",nrows=10**6 )
train_df

In [ ]:
#uniquの数を出そう
train_df.nunique()


In [ ]:
#uniqueの割合を確認
train_df['content_type_id'].value_counts().to_dict()

In [ ]:
#1ユーザーあたりのレコード数を出してみる
#多い人だと1万件のレコードが存在するようだ
train_df['user_id'].value_counts(sort=True,ascending=False)

In [ ]:
train_df['task_container_id'].value_counts(sort=True,ascending=False)

In [ ]:
train_df['prior_question_had_explanation'].value_counts(sort=True,ascending=False).to_dict()

In [ ]:
#content_idとtask_container_idの関係がよくわからないので調べてみる
#content_idとtask_container_idがイコール関係は100万行中168件のみ、少なくとも
#少なくともcontent_idとtask_container_idが同じ意味でないことがわかる
train_df[(train_df["content_id"] ==train_df['task_container_id'])]


In [ ]:
#ためしにcontent_id==1のデータに絞って出力してみよう
#content_id=1は、何種類かのtask_container_idに属していることがわかる
train_df[train_df["content_id"] ==1]


In [ ]:
#さらにtask_container_id==２０のデータに絞って出力してみよう
#task_container_idは、複数のcontent_idを持つことがわかった
train_df[train_df["task_container_id"] ==20].sort_values('user_id')

In [ ]:
#一人のユーザは、どのようなデータを持つのか確認しよう
#まず、レコード数(=質問の数が多い)が多いユーザを特定する
#とりあえずuser_id=1283420あたりをサンプルとする(id=7171715は、極端に値が大きい。異常値？)
user_df = train_df[train_df.answered_correctly != -1].groupby('user_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
user_df.columns = ['user_id', 'user_questions', 'user_mean']
user_df.sort_values('user_questions')

In [ ]:
#説明にある通り、１つのtask_container_idは、３つのcontent_idを持つことがわかった
train_df[train_df["user_id"] ==1283420]

In [ ]:
#CSV作成用に読み込み
train_df_csv = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv",nrows=1000)
train_df_csv.to_csv("train_df_1000.csv")

## カラムの説明
### row_id(int64)
>ID code for the row.

行に対するID

### timestamp(int64)
> the time in milliseconds between this user interaction and the first event completion from that user.

このユーザーインタラクション(ユーザの新規登録)から、該当レコードのイベント完了までのミリ秒単位の時間。
訓練データの１つのuser_idに着目すると、timestampがレコード発生ごとに増えていくのがわかるかと思います。


### user_id: (int32) 
> ID code for the user.

ユーザーのIDコード。

### content_id: (int16) 
> ID code for the user interaction

ユーザーインタラクションのIDコード。
「インタラクション」だと分かりづらい表現だから、単にコンテンツIDで良いと思う。



### content_type_id: (int8) 
> 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

イベントがユーザーに提示された質問の場合は0、イベントが講義を見ているユーザーの場合は1。
100万件のデータで比較すると、件数は以下のようになっている。
{0: 980093, 1: 19907}

### task_container_id: (int16) 
> Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

質問または講義のバッチのIDコード。
たとえば、ユーザーは、いずれかの説明を表示する前に、3つの質問を連続して表示する場合があります。
これら3つはすべてtask_container_idを共有します。

### user_answer: (int8) 
> the user's answer to the question, if any. Read -1 as null, for lectures.

質問に対するユーザーの回答(数字で答えるようだ)。講義の場合は、-1を「null」として読み取ります。
※質問した回数ではない(筆者は最初、質問の回数のことだと思っていた。)

### answered_correctly: (int8) 
> if the user responded correctly. Read -1 as null, for lectures.

ユーザーが正しく応答した場合。講義の場合は、-1をnullとして読み取ります。

### prior_question_elapsed_time: (float32) 
> The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

ユーザーが前の質問バンドルの各質問に回答するのにかかった平均時間（ミリ秒単位）。
その間の講義は無視されます。ユーザーの最初の質問バンドルまたは講義ではnullです。
時間は、ユーザーが前のバンドルの各質問を解決するのにかかった平均時間であることに注意してください。

### prior_question_had_explanation: (bool) 
> Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.


ユーザーが前の質問バンドルに回答した後、説明と正しい回答を見たかどうか。その間の講義は無視されます。
値は単一の質問バンドル間で共有され、ユーザーの最初の質問バンドルまたは講義ではnullになります。通常、ユーザーに表示される最初のいくつかの質問は、フィードバックが得られなかったオンボーディング診断テストの一部でした。

要するに、前回、質問に回答した後に解説をみたかどうか。最初の質問のときは、(当然のことながら)解説を見れないからnullになる。
実際の値は以下の通り。<br>
{True: 889887, False: 106297}


## questions.csv(質問データ)

In [ ]:
questions_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv")
questions_df

In [ ]:
#uniquの数を出そう
questions_df.nunique()


In [ ]:
questions_df['bundle_id'].value_counts(sort=True,ascending=False)

In [ ]:
#questions.csvを見ていると、question_idとbundle_idは、同じ値になっているように見える
#本当かどうか確認する
#確認した結果、同一bundle_idに、いくつかのquestion_idが紐づくことがわかる。
questions_df[questions_df["question_id"] != questions_df["bundle_id"]]

In [ ]:
#ためしに bundle_id=7795のデータを見てみよう
#大問題=bundle_id, 小問題=question_idと考えて良さそうだ
#partが全て一致、tagsもほぼ一致していることがわかる
questions_df[questions_df["bundle_id"] == 7795]

### question_id: 
> foreign key for the train/test content_id column, when the content type is question (0).

コンテンツタイプが質問ならば0、question_idは、train / testの「content_id」列の外部キー。


### bundle_id:
> code for which questions are served together.

質問が一緒に提供されるコード。
大問題=bundle_id, 小問題=question_idというイメージ。

### correct_answer:
> the answer to the question. Can be compared with the train user_answer column to check if the user was right.

質問への回答。 train.csvの「user_answer」列と比較して、ユーザーが正しいかどうかを確認できます

### part: 
> the relevant section of the TOEIC test.

TOEICテストの関連セクション。

### tags: 
> one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

質問の1つ以上の詳細なタグコード。タグの意味は提供されませんが、これらのコードは質問をまとめるのに十分です。


# lectures.csv(講義のメタデータ)


ユーザーが教育を進めるときに視聴する講義のメタデータ。

In [ ]:
lectures_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/lectures.csv")
lectures_df

### lecture_id: 
> foreign key for the train/test content_id column, when the content type is lecture (1).

コンテンツタイプが講義（1）の場合、train / testcontent_id列の外部キー。

### part:
>  top level category code for the lecture.

講義のトップレベルのカテゴリコード。

### tag:
>  one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

講義用の1つのタグコード。タグの意味は提供されませんが、これらのコードは講義をまとめるのに十分です。

### type_of:
>  brief description of the core purpose of the lecture

講義の主な目的の簡単な説明


## example_test_rows.csv(テストセットデータ)

時系列APIによって配信されるテストセットデータの3つのサンプルグループ。形式はtrain.csvとほぼ同じです。 AIチューターがいつでも実際に利用できる情報を反映する、2つの異なる列がありますが、一度に1人のユーザーの情報を厳密に表示するのではなく、APIパフォーマンスのためにユーザーインタラクションをグループ化します。一部のユーザーは、訓練データに表示されていない非表示のテストセットに表示され、Webサイトへの新着のモデリングにすばやく適応するという課題をエミュレートします。


In [ ]:
test_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_test.csv")
test_df

### prior_group_responses (string)
>provides all of the user_answer entries for previous group in a string representation of a list in the first row of the group. All other rows in each group are null. If you are using Python, you will likely want to call eval on the non-null rows. Some rows may be null, or empty lists.

prior_group_responses（string）は、前のグループのすべてのuser_answerエントリを、グループの最初の行のリストの文字列表現で提供します。各グループの他のすべての行はnullです。 Pythonを使用している場合は、null以外の行でevalを呼び出すことをお勧めします。一部の行はnullまたは空のリストである可能性があります。


### prior_group_answers_correct (string)
> provides all the answered_correctly field for previous group, with the same format and caveats as prior_group_responses. Some rows may be null, or empty lists.

prior_group_answers_correct（string）は、prior_group_responsesと同じ形式と警告で、前のグループのすべてのanswered_correctlyフィールドを提供します。一部の行はnullまたは空のリストである可能性があります。

## example_sample_submission.csv(提出サンプルファイル)

In [ ]:
submission_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv")
submission_df

# 時系列APIの詳細(Time-series API Details)

> Refer to the starter notebook for an example of how to complete a submission. The time-series API has changed somewhat from previous competitions!

提出を完了する方法の例については、[スターターノートブック](https://www.kaggle.com/sohier/competition-api-detailed-introduction)を参照してください。時系列APIは、以前のコンテストから多少変更されました。


> You should not try to submit anything for the rows that contain lectures.

講義を含む行には何も送信しようとしないでください。


> The API provides user interactions groups in the order in which they occurred. Each group will contain interactions from many different users, but no more than one task_container_id of questions from any single user. Each group has between 1 and 1000 users.

APIは、ユーザーインタラクショングループを発生順に提供します。各グループには、多くの異なるユーザーからのインタラクションが含まれますが、単一のユーザーからの質問のtask_container_idは1つだけです。各グループには1〜1000人のユーザーがいます。


> Expect to see roughly 2.5 million questions in the hidden test set.

非表示のテストセットで約250万の質問が表示されると予想されます。


> The API will load up to 1 GB of the test set data in memory after initialization. The initialization step (env.iter_test()) will require meaningfully more memory than that; we recommend you do not load your model until after making that call. The API will also consume roughly 15 minutes of runtime for loading and serving the data, but will also obfuscate the true runtime for all submissions.

APIは、初期化後に最大1GBのテストセットデータをメモリにロードします。初期化ステップ（env.iter_test（））は、それよりも意味のある量のメモリを必要とします。その呼び出しを行うまで、モデルをロードしないことをお勧めします。 APIは、データのロードと提供に約15分のランタイムも消費しますが、すべての送信の実際のランタイムを難読化します。

> The API loads the data using the types specified above (int32 for user_id, int8 for content_type_id, etc).

APIは、上記で指定されたタイプ（user_idの場合はint32、content_type_idの場合はint8など）を使用してデータをロードします。

